In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
%cd /content/drive/Othercomputers/Lenovo/BINUS/SKRIPSI/PRE THESIS/CODES

# %cd /content/drive/MyDrive/CODES

/content/drive/Othercomputers/Lenovo/BINUS/SKRIPSI/PRE THESIS/CODES


In [3]:
!ls

 01b_bertopic_cleaning_eda_preprocessing.ipynb
 01_cleansing_and_preprocessing.ipynb
 01c_sentiment_prediction_pipeline
 02_modeling_sentiment_prediction.ipynb
 03_modeling_topic_modeling.ipynb
 data_preparation.py
 df_modeling_BERT.csv
 df_modeling_bertopic.csv
 df_modeling.csv
 df_test2.csv
 df_test.csv
 df_train2.csv
 df_train.csv
 df_val.csv
 exclude_words.py
 mapping_dictionary.py
 negative_topic_modeling_results.xlsx
'Normalization Dictionary [Explained].gsheet'
 normalization_dictionary.py
 positive_topic_modeling_results.xlsx
 __pycache__
 results
 src
 stopwords.txt
 tes-bertopic.ipynb


# **Import Libraries and Prepare Dataset**

In [4]:
!pip install -q nlp_id stanza Sastrawi Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from wordcloud import WordCloud
from collections import Counter

import re
import nltk
nltk.download('punkt_tab')

from sklearn.model_selection import train_test_split

import stanza
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nlp_id.lemmatizer import Lemmatizer
from nlp_id.tokenizer import Tokenizer
from nlp_id.stopword import StopWord

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_colwidth", None)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
url = (
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQx0P59eV_KGGFJe-l86dgsz9pZRehTRJoOP_tyVXbrQPtzmD97E1lZ0lgFJ-ATVTT9HkBRX5g1kRKB/pub?output=csv"
)
df = pd.read_csv(url)
print(df.shape)

(301, 7)


In [7]:
df_melt = df.melt(
    value_vars=[
        "Apa yang Anda sukai selama masa perkuliahan Anda?",
        "Apa yang Anda tidak sukai selama masa perkuliahan Anda?"
    ],
    var_name='sentiment',
    value_name='text'
)

sentiment_map = {
    "Apa yang Anda sukai selama masa perkuliahan Anda?": 'positive',
    "Apa yang Anda tidak sukai selama masa perkuliahan Anda?": 'negative'
}
df_melt['sentiment'] = df_melt['sentiment'].map(sentiment_map)

df_preprocess = df_melt[['text', 'sentiment']]

# **Data Cleansing and Preprocessing**

## Split Data

In [8]:
def split_data(df, validation_data=False):
    if validation_data:
        train_df, val_test_df = train_test_split(
            df, test_size=0.2, random_state=42, stratify=df['sentiment']
        )
        val_df, test_df = train_test_split(
            val_test_df, test_size=0.5, random_state=42, stratify=val_test_df['sentiment']
        )

        X_train = train_df['text']
        y_train = train_df['sentiment']
        X_val = val_df['text']
        y_val = val_df['sentiment']
        X_test = test_df['text']
        y_test = test_df['sentiment']

        print(f'Training shape: {X_train.shape}, {y_train.shape}')
        print(f'Validation shape: {X_val.shape}, {y_val.shape}')
        print(f'Test shape: {X_test.shape}, {y_test.shape}')
        return X_train, y_train, X_val, y_val, X_test, y_test

    # Only train/test split
    train_df, test_df = train_test_split(
        df, test_size=0.2, random_state=42, stratify=df['sentiment']
    )
    X_train = train_df['text']
    y_train = train_df['sentiment']
    X_test = test_df['text']
    y_test = test_df['sentiment']
    print(f'Training shape: {X_train.shape}, {y_train.shape}')
    print(f'Test shape: {X_test.shape}, {y_test.shape}')
    return X_train, y_train, X_test, y_test

## Data Cleansing

In [9]:
from normalization_dictionary import norm_dict

def correct_typos(text):
    for typo, correction in norm_dict.items():
        text = re.sub(typo, correction, text, flags=re.IGNORECASE)
    return text

# Contoh: "Saya sukaaaa sekali dengan kampus saya" -> "Saya suka sekali dengan kampus saya"
def reduce_extra_characters(text):
    return re.sub(r'(.)\1{2,}', r'\1', text)

exception_words = ["tanya", 'punya', 'bertanya', 'hanya'] # Kata-kata yang berakhiran dengan -nya namun memiliki arti sendiri
def split_nya(word):
    if word in exception_words:
        return word
    return re.sub(r'(.*?)nya$', r'\1 nya', word)

def process_split_nya(review):
    words = review.split()
    processed_words = [split_nya(word.strip()) for word in words]
    return ' '.join(processed_words)

def text_cleansing(text):
    # Mengubah ke huruf kecil
    text = text.lower()

    # Menghapus simbol dan angka
    text = re.sub(r"[^a-zA-Z\s']", ' ', text)

    # Memisahkan kata dengan akhiran -nya
    text = process_split_nya(text)

    # Memperbaiki typo dan huruf yang berlebih
    text = correct_typos(text)
    text = reduce_extra_characters(text)

    # Menghapus spasi yang berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text

## Data Preprocessing

In [10]:
from exclude_words import exclude_lemmatization, exclude_stopwords

lemmatizer = Lemmatizer()
tokenizer = Tokenizer()
stopword = StopWord()

# Menambahkan kata untuk stop words
stop_words = stopword.get_stopword()
custom_stopwords = ['nya', 'ya', 'nih']
stop_words.append(custom_stopwords)

def text_preprocessing(text):
    # Tokenisasi menggunakan Tokenizer dari nlp_id
    tokens = tokenizer.tokenize(text)

    # Lematisasi dan hapus stopwords
    lemmatized_tokens = [
        lemmatizer.lemmatize(word) if (word not in exclude_stopwords and word not in exclude_lemmatization) else word
        for word in tokens
        if word not in stop_words or word in exclude_stopwords
    ]

    # Gabungkan kembali menjadi teks dan hapus spasi berlebih
    processed_text = re.sub(r'\s+', ' ', ' '.join(lemmatized_tokens)).strip()

    return processed_text

In [11]:
from exclude_words import exclude_lemmatization, exclude_stopwords

stanza.download('id')
nlp = stanza.Pipeline(lang='id', processors='tokenize,mwt,pos,lemma')

factory = StopWordRemoverFactory()
sastrawi_stopwords = set(factory.get_stop_words())
custom_stopwords = {'nya', 'ya', 'nih'}
all_stopwords = sastrawi_stopwords.union(custom_stopwords)

def text_preprocessing2(text):
  doc = nlp(text)

  lemmatized_tokens = []
  for sentence in doc.sentences:
      for word in sentence.words:
          token = word.text.lower()
          lemma = word.lemma.lower()

          # Lakukan filtering stopwords dan pengecualian
          if token in exclude_stopwords or token not in all_stopwords:
              lemmatized_token = token if token in exclude_lemmatization else lemma
              lemmatized_tokens.append(lemmatized_token)

  # Hapus spasi berlebih dan gabungkan kembali ke teks
  cleaned_text = re.sub(r'\s+', ' ', ' '.join(lemmatized_tokens)).strip()

  return cleaned_text

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: id (Indonesian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/id/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| pos       | gsd_charlm   |
| lemma     | gsd_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


## Vectorize Text

## Data Preparation

In [12]:
def prepare_dataset(df, model_type, preprocess_type=None):
  df['cleaned_text'] = df['text'].apply(text_cleansing)

  if model_type == 'svm':
    if preprocess_type == 'nlp_id':
      df['preprocessed_text_nlp_id'] = df['cleaned_text'].apply(text_preprocessing)
      return df

    if preprocess_type == 'stanza':
      df['preprocessed_text_stanza'] = df['cleaned_text'].apply(text_preprocessing2)
      return df

  if model_type == 'indobert':
    return df

# **Modeling**

In [13]:
pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nlp-id 0.1.19.0 requires huggingface-hub==0.31.1, but you have huggingface-hub 0.34.3 which is incompatible.


In [14]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import svm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

# **PIPELINE**

In [15]:
import pickle

with open('./src/vectorizer1.pkl', 'rb') as f:
    vectorizer1 = pickle.load(f)

with open('./src/model_svm1.pkl', 'rb') as f:
    model1 = pickle.load(f)

with open('./src/vectorizer2.pkl', 'rb') as f:
    vectorizer2 = pickle.load(f)

with open('./src/model_svm2.pkl', 'rb') as f:
    model2 = pickle.load(f)

model_indobert = AutoModelForSequenceClassification.from_pretrained("./src/indobert_model")
tokenizer_indobert = AutoTokenizer.from_pretrained("./src/indobert_tokenizer")

In [16]:
def run_svm_pipeline(df, model_type, vectorizer=None, model=None, preprocess_type=None):
  start_time = time.time()
  df_preprocessed = prepare_dataset(df, model_type=model_type, preprocess_type=preprocess_type)

  df_tfidf = vectorizer.transform(df_preprocessed[f'preprocessed_text_{preprocess_type}'])
  tfidf_test_df = pd.DataFrame(df_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

  y_pred = model.predict(tfidf_test_df)

  end_time = time.time()
  execution_time = end_time - start_time
  print(f"Execution time for {model_type} using {preprocess_type}: {execution_time} seconds")
  return y_pred

In [41]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def run_indobert_pipeline(df, model, tokenizer):
  start_time = time.time()

  model.eval()

  df_preprocessed = prepare_dataset(df, model_type='indobert')
  texts = df_preprocessed['cleaned_text'].tolist()

  cleaned_texts = [text_cleansing(t) for t in texts]
  inputs = tokenizer(cleaned_texts, return_tensors='pt', padding=True, truncation=True)

  with torch.no_grad():
      outputs = model(**inputs)
      preds = torch.argmax(outputs.logits, dim=1)
      labels = [id2label[int(p)] for p in preds]

  end_time = time.time()
  execution_time = end_time - start_time
  print(f"Execution time for IndoBERT: {execution_time} seconds")
  return labels

## Inference

In [44]:
label_map = {'positive':1, 'negative':0}
id2label = {v: k for k, v in label_map.items()}
target_names = ['negative', 'positive']

### **SVM**

In [32]:
X_train, y_train, X_test, y_test = split_data(df_preprocess, validation_data=False)
y_train = y_train.map(label_map)
y_test = y_test.map(label_map)

df_test = pd.DataFrame({'text': X_test, 'sentiment': y_test})

y_pred_test_nlp_id = run_svm_pipeline(df_test, model_type='svm', vectorizer=vectorizer1, model=model1, preprocess_type='nlp_id')
y_pred_test_stanza = run_svm_pipeline(df_test, model_type='svm', vectorizer=vectorizer2, model=model2, preprocess_type='stanza')

Training shape: (481,), (481,)
Test shape: (121,), (121,)
Execution time for svm using nlp_id: 0.7618815898895264 seconds
Execution time for svm using stanza: 13.362156629562378 seconds


In [35]:
print("Classification Report for SVM with nlp_id:")
print(classification_report(y_test, y_pred_test_nlp_id, target_names=target_names))

Classification Report for SVM with nlp_id:
              precision    recall  f1-score   support

    negative       0.92      0.95      0.94        61
    positive       0.95      0.92      0.93        60

    accuracy                           0.93       121
   macro avg       0.93      0.93      0.93       121
weighted avg       0.93      0.93      0.93       121



In [36]:
print("Classification Report for SVM with stanza:")
print(classification_report(y_test, y_pred_test_stanza, target_names=target_names))

Classification Report for SVM with stanza:
              precision    recall  f1-score   support

    negative       0.97      0.97      0.97        61
    positive       0.97      0.97      0.97        60

    accuracy                           0.97       121
   macro avg       0.97      0.97      0.97       121
weighted avg       0.97      0.97      0.97       121



### **IndoBERT**

In [45]:
x_train, y_train, x_val, y_val, x_test, y_test = split_data(df_preprocess, validation_data=True)

df_test_indobert = pd.DataFrame({'text': x_test, 'sentiment': y_test})

y_pred_indobert = run_indobert_pipeline(df_test_indobert, model_indobert, tokenizer_indobert)

Training shape: (481,), (481,)
Validation shape: (60,), (60,)
Test shape: (61,), (61,)
Execution time for IndoBERT: 9.968843936920166 seconds


In [47]:
print("Classification Report for IndoBERT:")
print(classification_report(y_test, y_pred_indobert))

Classification Report for IndoBERT:
              precision    recall  f1-score   support

    negative       0.97      1.00      0.98        31
    positive       1.00      0.97      0.98        30

    accuracy                           0.98        61
   macro avg       0.98      0.98      0.98        61
weighted avg       0.98      0.98      0.98        61



# **Summary**

## **SVM**
SVM model used two different preprocessing methods and TF-IDF Vectorizer:
- nlp_id for lemmatization and stopwords removal -> achieved **93%** accuracy
- stanza for lemmatization and Sastrawi for stopwords removal -> achieved **97%** accuracy

Whilehe accuracy is both good and not that different, the time execution between both model showed a bigger gap in predicting the test set. SVM using nlp_id predicted in **under 1 second**, while SVM using stanza and Sastrawi predicted in **13 seconds**.

With a similar time execution, it is better to use SVM with nlp_id because it's faster than SVM with stanza and Sastrawi.

## **IndoBERT**
IndoBERT model achieved higher accuracy than both SVM models, with **98%** accuracy which is expected since it is a larger model. But the time execution and architecture is surely bigger than traditional machine learning models. With a time execution of almost **10 seconds**, we can reconsider to use machine learning model instead of transformers to reduce the computational cost
